In [1]:
import numpy as np
import sympy as sp
from plotly import graph_objects as go
import script as rt
import math
from pyquaternion import Quaternion
import quaternion

In [9]:
h_slam_points = np.array([[1, 1, 1, 1], [2, 2, 2, 1], [4, 4, 4, 1], [6, 6, 6, 1]])
h_sensor_points = np.array([[-1, -1, -1, 1], [-2, -2, -2, 1], [-4, -4, -4, 1], [-6, -6, -6, 1]])
#h_sensor_points = np.array([[11, 1, 1, 1], [8, 4, 4, 1], [5, 6, 5, 1], [4, 8, 6, 1], [4, 12, 7, 1]]) 
# so vtorite tocki ne raboti dobro
x_se = h_sensor_points[:, 0]
y_se = h_sensor_points[:, 1]
z_se = h_sensor_points[:, 2]
x_sl = h_slam_points[:, 0]
y_sl = h_slam_points[:, 1]
z_sl = h_slam_points[:, 2]
xsr = np.mean(x_se)
ysr = np.mean(y_se)
zsr = np.mean(z_se)
xSr = np.mean(x_sl)
ySr = np.mean(y_sl)
zSr = np.mean(z_sl)

In [10]:
def translation(x, y, z, xsred, ysred, zsred):
    i = 0
    xk = []
    yk = []
    zk = []
    xsum = 0 
    ysum = 0
    zsum = 0
    for i in range(len(x)):
        A = np.array([[x[i]],
                [y[i]],
                [z[i]], 
                [1]])
        B = np.array([[1, 0, 0, -xsred],
                [0, 1, 0, -ysred],
                [0, 0, 1, -zsred], 
                [0, 0, 0, 1]])
        C = B @ A
        x1 = C[0][0]
        y1 = C[1][0]
        z1 = C[2][0]
        xsum = xsum + x1
        ysum = ysum + y1
        zsum = zsum + z1
        xk.append(x1)
        yk.append(y1)
        zk.append(z1)
        i = i + 1
    tsredx = xsum/len(x)
    tsredy = ysum/len(y)
    tsredz = zsum/len(z)
    return xk, yk, zk, tsredx, tsredy, tsredz

xP, yP, zP, tsredx, tsredy, tsredz = translation(x_se, y_se, z_se, xsr, ysr, zsr) # translacija na tockite od senzori


xS, yS, zS, tsredxS, tsredyS, tsredzS = translation(x_sl, y_sl, z_sl, xSr, ySr, zSr)  # translacija na tockite od SLAM

In [11]:
def suma_rastojanija(x,y,z,xs,ys,zs):
    D = 0
    for i in range(len(x)):
        D = D + math.sqrt((x[i] - xs)**2 + (y[i] - ys)**2 + (z[i] - zs)**2)
    return D
d = suma_rastojanija(xP, yP, zP, tsredx, tsredy, tsredz)
dS = suma_rastojanija(xS, yS, zS, tsredxS, tsredyS, tsredzS)
scal = d /dS
print(scal)

1.0


In [12]:
def scale(x, y, z, sx, sy, sz):
    xsk, ysk, zsk = [], [], []
    i = 0
    for i in range(len(x)):
        A = np.array([[x[i]],
                [y[i]],
                [z[i]], 
                [1]])

        B = np.array([[sx, 0, 0, 0],
                    [0, sy, 0, 0],
                    [0, 0, sz, 0],
                    [0, 0, 0, 1]])
        C = B@A
        xs = C[0][0]
        ys = C[1][0]
        zs = C[2][0]
        xsk.append(xs)
        ysk.append(ys)
        zsk.append(zs)
        i = i + 1
    
    return xsk, ysk, zsk

xsk, ysk, zsk = scale(xS, yS, zS, scal, scal, scal)

In [13]:
w = np.ones(len(xP))
sensor_data = rt.hpoint3(xP, yP, zP, w)
sensor_data = np.array(sensor_data, dtype=np.float)

In [14]:
slam_scale = rt.hpoint3(xsk, ysk, zsk, w)
slam_scale = np.array(slam_scale, dtype=np.float)

In [15]:
fig = go.FigureWidget()
#fig.update_layout(xaxis={'range': [-7, 6]}, yaxis={'range': [1, 13]}),
fig.update_layout(
    scene = dict(
        xaxis = dict(),
                     yaxis = dict(),
                     zaxis = dict(),),)
fig.add_trace(go.Scatter3d(x=sensor_data[0, :], y=sensor_data[1, :], z=sensor_data[2, :], mode='markers', name='Сензори',
                         marker={'size': 5, 'symbol': 'x', 'color': np.arange(sensor_data.shape[1])}))
fig.add_trace(go.Scatter3d(x=slam_scale[0, :], y=slam_scale[1, :], z=slam_scale[2, :], mode='markers', name='SLAM', 
                          marker={'size': 5, 'color': np.arange(slam_scale.shape[1])}))

fig

FigureWidget({
    'data': [{'marker': {'color': array([0, 1, 2, 3]), 'size': 5, 'symbol': 'x'},
             …

In [16]:
def error(sensor_points, slam_points, alpha, beta, gama):
    """
    Функција која ја пресметува грешката, на начин на кој сме ја дефинирале.
    """
    transform = rt.hrotation3('x', alpha) * rt.hrotation3('y', beta) * rt.hrotation3('z', gama)
    transform = np.array(transform, dtype=np.float)
    new_sensor_points = (transform @ slam_points)
    e = sensor_points.T - new_sensor_points.T
    return np.apply_along_axis(lambda x: x.T @ x, 1, e)

def step_gradient(sensor_points, slam_points, transform, learning_rate):
    """
    Чекор со gradient descent.
    
    h_sensor_points, h_slam_points: се податоците кои ќе ни послужат за пресметка на грешката.
    transform: е матрицата на трансформација чии членови ги бараме во оваа функција
    learning_rate: стапката на учење
    """
    x_gradient, y_gradient, z_gradient, alpha_gradient, beta_gradient, gama_gradient = 0, 0, 0, 0, 0, 0
    new_sensor_points = (transform @ slam_points).T
    r_vector = sensor_points.T - new_sensor_points 
    r_x_vector, r_y_vector, r_z_vector = r_vector[:, 0], r_vector[:, 1], r_vector[:, 2]  
    m = len(r_vector)
    for r_x, r_y, r_z, sensor_point in zip(r_x_vector, r_y_vector, r_z_vector, sensor_points):
        x_gradient += r_x 
        y_gradient += r_y  
        z_gradient += r_z
        alpha_gradient += -sensor_point[1] * r_x + sensor_point[0] * r_y   
        beta_gradient += sensor_point[2] * r_x - sensor_point[0] * r_z     
        gama_gradient += -sensor_point[2] * r_y + sensor_point[1] * r_z    
    dx = -learning_rate/10 * x_gradient / m  
    dy = -learning_rate/10 * y_gradient / m  
    dz = -learning_rate/10 * z_gradient / m
    dalpha = -learning_rate * alpha_gradient / m   
    dbeta = -learning_rate * beta_gradient / m
    dgama = -learning_rate * gama_gradient / m
    # претходните 3 чекори по секој степен на слобода ги соединуваме во една трансформациона матрица
    delta_transform = np.array(rt.hrotation3('x', dalpha) * rt.hrotation3('y', dbeta) * rt.hrotation3('z', dgama), dtype=np.float)
    # и конечно го правиме чекорот
    transform = delta_transform @ transform
    #new_sensor_points = (transform @ slam_points)
    # на крајот ја враќаме обновената вредност на transform и предвидената вредност за изместените податоци за да ги исцртаме
    return transform, new_sensor_points

def run(fig, sensor_points, slam_points):
    learning_rate = -0.01
    num_iterations = 10000
    # почнуваме со случајно избрани вредности за x, y, и alpha
    alpha, beta, gama = 0, 0, 0
    transform = np.array(rt.hrotation3('x', alpha) * rt.hrotation3('y', beta) * rt.hrotation3('z', gama), dtype=np.float)
    for i in range(num_iterations):
        transform, new_sensor_points = step_gradient(sensor_points, slam_points, transform, learning_rate)
        # цртаме на секоја стота итерација
        if i % 100 == 0:
            alpha, beta, gama = np.arctan2(transform[1, 0], transform[0, 0]) / np.pi * 180, np.arctan2(transform[0, 1], transform[1, 1]) / np.pi * 180, np.arctan2(transform[0, 2], transform[2, 2]) / np.pi * 180
            with fig.batch_update():
                fig.data[1].x, fig.data[1].y, fig.data[1].z = new_sensor_points[:, 0], new_sensor_points[:, 1], new_sensor_points[:, 2]
                fig.data[1].marker.color = np.arange(len(new_sensor_points))
    
    alpha, beta, gama = np.arctan2(transform[1, 0], transform[0, 0]) / np.pi * 180, np.arctan2(transform[0, 1], transform[1, 1]) / np.pi * 180, np.arctan2(transform[0, 2], transform[2, 2]) / np.pi * 180
    return alpha, beta, gama

run(fig, sensor_data, slam_scale)

(176.40801681616531, -138.7625434718797, -28.47709083528058)